# Cyberbullying Classification

### 1. The degree of aggressivity & Negativity

### 2. Personnalization
    - Depency tree, establish the subject, object, complement, aux
    - Also, tell the (person 1st, 2nd, 3rd) of the subject and the object. 
    - Identify what are they accuse from 
    - Identify who’s being targeted (Lexicon or wordnet)
    - Established if the aux is directed to the object  (Establishing intention)
### 3. Harms inflicted and to who
    - Now, using the complement (output of dependency tree), we can map these to threats
    - We need wordnet to identify threat. (Actually better lexicon than word net)
    - To object refers to what category? race, nationality, religion, color, gender, 



Set your environment
- source spacy/bin/activate (Bash)
- get back to your base: deactivate (Bash)

You have to be in anaconda3 interpreter

# Import data

In [200]:
import numpy as np
import pandas as pd 
import os
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
import re
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [201]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import nltk
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()

macbook = ('/Users/philippebeliveau/Desktop/Notebook/Winter_2024/Text_mining/Git_MiningRepository/Text_miningProject/Notebook/cyberbullying_tweets.csv')

mac_mini = ('/Users/philippebeliveau/Desktop/Notebook_Jupyter_R/Winter_2024/Text_mining/Project/Text_miningProject/Notebook/cyberbullying_tweets.csv')

df = pd.read_csv(macbook)
df.head()
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [202]:
# Initialize an empty dictionary to hold the tweets by category
tweets_by_category = {}

# Iterate over each category
for category in categories:
    # Filter the dataset for the current category
    category_tweets = df[df['cyberbullying_type'] == category]['tweet_text'].tolist()
    
    # Add the list of tweets to the dictionary
    tweets_by_category[category] = category_tweets

category = tweets_by_category['ethnicity'][100:120]

# Data preprocessing

In [203]:
import re

def clean_text(text):
    #removing hastags and links
    pattern=re.compile(r"(#[a-zA-Z0-9]+|@[a-zA-Z0-9]+|https?://\S+|www\.\S+|\S+\.[a-z]+|RT @)")
    text = pattern.sub('', text)
    text = " ".join(text.split())
    
    return text

In [204]:
#cleaning of data
df['clean_data']=df['tweet_text'].apply(lambda x: clean_text(x))

In [205]:
df.sample(10)

,tweet_text,cyberbullying_type,clean_data
9051,VIDEO: Miley Cyrus Makes Date Rape Joke at Gay Nightclub... http://ift.tt/1iDWpO6 #politics,gender,VIDEO: Miley Cyrus Makes Date Rape Joke at Gay Nightclub...
44960,"Lmaoo ""@stransinf: @_WSBG fuck u hatin on budden for u dumb nigger""",ethnicity,"Lmaoo "": @_WSBG fuck u hatin on budden for u dumb nigger"""
2297,The only thing sneakier than a meatball sandwich is a BBQ pork sub.,not_cyberbullying,The only thing sneakier than a meatball sandwich is a BBQ pork sub.
30894,"@jaredchase everything. right now, i'm mostly in love wiht my ele sham/spriest (but disc for 2s)",other_cyberbullying,"everything. right now, i'm mostly in love wiht my ele sham/spriest (but disc for 2s)"
42443,"Thank you, I become upset, I can see the outcome. It was LBJ's prized Great Society, however, it was a ploy to get Black people to vote for him. He was a wiley, disgusting man who literally stated, ""We will have those Niggers' voting Democrat for rhe next 200 years! """,ethnicity,"Thank you, I become upset, I can see the outcome. It was LBJ's prized Great Society, however, it was a ploy to get Black people to vote for him. He was a wiley, disgusting man who literally stated, ""We will have those Niggers' voting Democrat for rhe next 200 years! """
13204,NY Post Makes Gay Rape Joke on Jared Fogle Cover http://twib.in/l/kKaeAanr5bL #LGBT |,gender,NY Post Makes Gay Rape Joke on Jared Fogle Cover |
23367,"That’s one of trump’s oldest lies, spoken right out by a “Christian woman” who said she’d never lie.",religion,"That’s one of trump’s oldest lies, spoken right out by a “Christian woman” who said she’d never lie."
9777,"@StJ51 the same way that Rape jokes, and Race jokes, and Holocaust joke, and Gay jokes are in bad taste too? #ThinkAboutIt",gender,"the same way that Rape jokes, and Race jokes, and Holocaust joke, and Gay jokes are in bad taste too?"
7714,"You haven't actually ""sold out"" if you're not charging for the food.",not_cyberbullying,"You haven't actually ""sold out"" if you're not charging for the food."
21530,"Only if you guys read Quran and teaching of Prophet Muhammad SAW then you guys will know how peaceful Islam is. People just see ISIS, Al Qaeda and 9/11 and think that is Islam. Its a BIG NO ! Those are brainwashed TERRORISTS who have no religion.",religion,"Only if you guys read Quran and teaching of Prophet Muhammad SAW then you guys will know how peaceful Islam is. People just see ISIS, Al Qaeda and 9/11 and think that is Islam. Its a BIG NO ! Those are brainwashed TERRORISTS who have no religion."


In [206]:
# Initialize an empty dictionary to hold the tweets by category
tweets_by_category = {}

# Iterate over each category
for category in categories:
    # Filter the dataset for the current category
    category_tweets = df[df['cyberbullying_type'] == category]['clean_data'].tolist()
    
    # Add the list of tweets to the dictionary
    tweets_by_category[category] = category_tweets

category = tweets_by_category['ethnicity'][100:120]

# Part 2 - Critique vs Insult

In [207]:
# Example 
en_EntNom = "Julie, the arabs are the ones that are going to push the real change. they are the ones in power, generally. we need their help."
ent1 = "oh look it's that dumb niggers ghost acct! Fuck you you dumb motherfucker!"
#On charge le texte dans le format  de Spacy et on utilise la fonction de visualisation displacy pour surligné les ENs
doc = nlp(ent1)
displacy.render(doc, style="ent")

/Users/philippebeliveau/miniforge3/lib/python3.10/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


# Simple approach
- Extract the adjectives, nouns, the Subject Person, and the complement/conjunction. 
- Extract the name entities
- Extract all the synonyms of the adjective, nouns and complement into a long list.
- Create a polarity score using Vadersentiment on the tweet itself and on the list of synonyms

In [208]:
Gender = ["rape is real..zvasiyana nema jokes about being drunk or being gay or being lesbian...rape is not ones choice or wish..thtz where the sensitivity is coming from",
         "You never saw any celebrity say anything like this for Obama: B Maher Incest Rape 'Joke' S Colbert Gay 'joke' K Griffin beheading 'joke'",
         "@ManhattaKnight I mean he's gay, but he uses gendered slurs and makes rape jokes",
         "RT @Raul_Novoa16: @AliciaBernardez @Alex_Aim @_mecaesmal feminazi",
         "Rape is rape. And the fact that I read one post about a guy getting raped and the comments are calling him gay and he should be happy...? stfu and I really hope no one takes this as a joke tf you own no ones body. You have no rights to do whatever you want to someone else.",
         "@coiny Also, it's hard to take a company seriously re: harassment when the game has rape jokes & gay jokes and treats women the way it does.",
         "Idgaf if you are gay, lesbian, bisexual, or whatever the fuck you are, you don't fuck make a rape joke and think it's fucken ok",
         "#GermanProfessor gives meaning to term FemiNazi. Doesn't hire 'Indian Men' bcz of 'Indias's Rape Problem'",
         "RT @mcclure111: #DontDateSJWs #ThatWouldBeAViolationOfTheRestrainingOrder",
         "So I call you female I’m basically calling you a bitch ?"]

Ethnicity = [
"Hey dumb fuck celebs stop doing something for people for publicity on Facebook... Wtf happen to life u niggers are cowards.",
"Fuck u bitch RT @tayyoung_: FUCK OBAMA, dumb ass nigger",
"@JoeBiden No Joe, YOU are the RACIST. They have so many video and audio clips of the RACIST shit you have spewed in your past, How can anyone see you as anything but a RACIST. You hate Black people just admit it. And you loves the women to touch. Just look at your own stuff!",
"when your truck looks dumb as fuck out trying to make it look worse. you ain't country. quit tryin nigger.",
"That nigger food in the cafe today was disgusting",
"Muthafuckas still EGO tripping!Fool fuck your damn ego and live life to the fullest!!! LOL at these dumb ass niggers with the bullshit!",
"@iMADE_YouMOAN don't play dumb, I should fuck yo little white ass up you nigger. 3chainz.? Aw.",
"I WILL ONLY SAY IT ONCE! IF YOU DO NOT FEAR HAVE THE SUSPICION OF BEING CALLED A NIGGER! NIGGA! OR NEGRO AT ANY POINT IN YOUR LIFE YOU CANNOT USE THE N WORD. PERIOD. I am talking about non black people in specific!!!! White passing mixed kids are on THIN ICE.",
"@CJ_Jacobs oh look it's that dumb niggers ghost acct! Fuck you you dumb motherfucker!",
"@Otoliver @Yo_PaTnA_dEm20 this bitch is dumb talk bout 'do not dew 'said not sayed' nigger not nigga lol bitch fuck off"
]

noncyber = [
"In other words #katandandre, your food was crapilicious! #mkr",
"Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc",
"@XochitlSuckkks a classy whore? Or more red velvet cupcakes?",
"@Jason_Gio meh. :P  thanks for the heads up, but not too concerned about another angry dude on twitter.",
"@RudhoeEnglish This is an ISIS account pretending to be a Kurdish account.  Like Islam, it is all lies.",
"@Raja5aab @Quickieleaks Yes, the test of god is that good or bad or indifferent or weird or whatever, it all proves gods existence.",
"Itu sekolah ya bukan tempat bully! Ga jauh kaya neraka",
"Karma. I hope it bites Kat on the butt. She is just nasty. #mkr",
"@stockputout everything but mostly my priest",
"Rebecca Black Drops Out of School Due to Bullying:"
]


In [209]:
# Initialize lists to store data
data = []

for tweet in df['clean_data']:
    # Analyze the tweet
    doc = nlp(tweet)

    # Extract adjectives, nouns, subjects and complements/conjunctions
    adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    subjects = [(token.text, token.morph.get('Person')) for token in doc if token.dep_ == "nsubj"]
    complements = [token.text for token in doc if token.dep_ in ("acomp", "conj", "ROOT")]

    # Extract children of ccomp or conj
    ccomp_conj_children = []
    ccomp_conj_sentiments = []
    for token in doc:
        if token.dep_ in ("ccomp", "conj", 'ROOT'):
            children = [child.text for child in token.children]
            ccomp_conj_children.append(children)
            # Compute sentiment scores for each child and append to list
            ccomp_conj_sentiments.extend([analyzer.polarity_scores(child)["compound"] for child in children])

    # Compute sum of sentiment scores for ccomp or conj children
    sum_ccomp_conj_sentiment = sum(ccomp_conj_sentiments)

    # Extract named entities
    named_entities = [ent.text for ent in doc.ents]

    # Extract synonyms
    synonyms = []
    for token in doc:
        if token.pos_ in ("ADJ", "NOUN", 'ccomp'):
            for syn in wordnet.synsets(token.text):
                for lemma in syn.lemmas():
                    synonyms.append(lemma.name())

    # Create a polarity score
    sentiment_score = analyzer.polarity_scores(tweet)["compound"]
    synonyms_sentiment_score = analyzer.polarity_scores(" ".join(synonyms))["compound"]

    # Append data to list
    data.append({
        "Tweet": tweet,
        "Adjectives": adjectives,
        "Nouns": nouns,
        "Subjects": subjects,
        "Complements": complements,
        "CCOMP or CONJ Children": ccomp_conj_children,
        "CCOMP or CONJ Children Sum Sentiment": sum_ccomp_conj_sentiment,
        "Named Entities": named_entities,
       # "Synonyms": synonyms,
        "Sentiment Score": sentiment_score,
        "Synonyms Sentiment Score": synonyms_sentiment_score
    })

# Create DataFrame
df = pd.DataFrame(data)

In [212]:
def contains_person(subjects, person):
    person_list = [p for _, p_list in subjects for p in p_list]
    return str(person) in person_list

df['Has 1st Person'] = df['Subjects'].apply(lambda x: contains_person(x, 1))
df['Has 2nd Person'] = df['Subjects'].apply(lambda x: contains_person(x, 2))
df['Has 3rd Person'] = df['Subjects'].apply(lambda x: contains_person(x, 3))

df['Word Count'] = df['Tweet'].apply(lambda x: len(x.split()))
df['Adjusted Sentiment Score'] = df.apply(lambda row: row['CCOMP or CONJ Children Sum Sentiment'] / len(row['CCOMP or CONJ Children']) if len(row['CCOMP or CONJ Children']) != 0 else 0, axis=1)
df['Has Capital Word'] = df['Tweet'].apply(lambda x: any(word.isupper() for word in x.split()))


### Save df

In [213]:
df.to_csv('/Users/philippebeliveau/Desktop/Notebook/Winter_2024/Text_mining/Dataset/v1.csv')

# Classification

In [214]:
def classify(row):
    education_terms = ['school', 'schools', 'college', 'graduation']
    if any(term in word for term in education_terms for word in row['Nouns']):
        return 'Critique/Other'
    elif row['Adjusted Sentiment Score'] > -0.12:
        return 'Critique/Other'
    elif row['Adjusted Sentiment Score'] < -0.08 and row['Sentiment Score'] > 0.5 and row['Synonyms Sentiment Score'] > 0.2:
        return 'Critique/Other'
    elif row['Has Capital Word'] == True and row['Adjusted Sentiment Score'] < -0.03 and 'you' in row['Subjects']:
        return 'Insults'
    else:
        return 'Insults'

df['Classification'] = df.apply(classify, axis=1)
df

,Tweet,Adjectives,Nouns,Subjects,Complements,CCOMP or CONJ Children,CCOMP or CONJ Children Sum Sentiment,Named Entities,Sentiment Score,Synonyms Sentiment Score,Has 1st Person,Has 2nd Person,Has 3rd Person,Word Count,Adjusted Sentiment Score,Has Capital Word,Classification
0,"In other words , your food was crapilicious!","[other, crapilicious]","[words, food]","[(food, [])]","[was, crapilicious]","[[In, ,, food, crapilicious, !]]",0.0000,[],0.0000,0.4767,False,False,False,8,0.000000,False,Critique/Other
1,Why is so white?,[white],[],[],"[is, white]","[[Why, white, ?]]",0.0000,[],0.0000,-0.2023,False,False,False,4,0.000000,False,Critique/Other
2,a classy whore? Or more red velvet cupcakes?,"[classy, more, red]","[whore, velvet, cupcakes]",[],"[whore, cupcakes]","[[a, classy, ?], [Or, velvet, ?]]",0.4404,[],-0.4137,-0.9702,False,False,False,8,0.220200,False,Critique/Other
3,"_Gio meh. :P thanks for the heads up, but not too concerned about another angry dude on twitter.","[concerned, angry]","[meh, thanks, heads, dude, twitter]",[],"[_, thanks, concerned]","[[meh, .], [:P, for, ,, but, concerned, .], [not, too, about]]",0.1728,[Gio meh],-0.4183,-0.6369,False,False,False,18,0.057600,True,Critique/Other
4,"This is an ISIS account pretending to be a Kurdish account. Like Islam, it is all lies.",[Kurdish],"[account, account, lies]","[(This, []), (it, [3])]","[is, is]","[[This, account, .], [Like, ,, it, lies, .]]",-0.0603,"[ISIS, Kurdish, Islam]",0.0258,0.1280,False,False,True,17,-0.030150,True,Critique/Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47687,"Black ppl aren't expected to do anything, depended on for anything. Yet free to participate, work, enjoy the freedom of humans all across this globe. If you waste your energy on the past you will be wasting it for building the future.","[Black, free]","[ppl, freedom, humans, globe, energy, past, future]","[(you, [2]), (you, [2])]","[expected, depended, free, work, enjoy, wasting]","[[ppl, are, n't, do, depended, .], [on, for], [Yet, participate, ,, work, .], [,, enjoy], [freedom], [waste, you, will, be, it, for, .]]",0.7093,[],0.8074,0.9978,False,True,False,42,0.118217,False,Critique/Other
47688,Turner did not withhold his disappointment. Turner called the court an “abominable conclave of negro hating demons” (with one exception) who “issued another decree that colored men and women must be driven into Jim Crow cars whenever it suits the whim of any white community.”,"[abominable, colored, white]","[disappointment, court, conclave, negro, demons, exception, decree, men, women, cars, whim, community]","[(Turner, []), (Turner, []), (who, []), (it, [3])]","[withhold, called, women]","[[Turner, did, not, disappointment, .], [Turner, court, conclave, ., ”], []]",-0.5106,"[Turner, Turner, one, Jim Crow]",-0.1526,-0.9747,False,False,True,45,-0.170200,False,Insults
47689,I swear to God. This dumb nigger bitch. I have got to bleach my hair reeeeeal fuckin' soon. D:&lt; FUCK.,"[dumb, D:&lt]","[nigger, bitch, hair]","[(I, [1]), (I, [1])]","[swear, bitch, got, D:&lt]","[[I, to, .], [This, dumb, nigger, .], [I, have, bleach, .], [;, FUCK, .]]",-1.7015,[],-0.9483,-0.9933,True,False,False,20,-0.425375,True,Insults
47690,"Yea fuck you therealexel: IF YOURE A NIGGER FUCKING UNFOLLOW ME, FUCKING DUMB NIGGERS.","[Yea, NIGGER, FUCKING]","[fuck, therealexel, UNFOLLOW, DUMB, NIGGERS]",[],[fuck],"[[Yea, you, therealexel, :, ME, .]]",0.0000,[],-0.9392,-0.9928,False,False,False,14,0.000000,True,Critique/Other


# Evaluation

In [216]:
# View result
pd.set_option('display.max_colwidth', None)
#df.drop(columns=['Synonyms'])

In [218]:
import pandas as pd

pd.set_option('display.max_rows', 10)

sample_df = df[['Tweet', 'Classification']].sample(100)
sample_df


,Tweet,Classification
28361,"hmmm. That is a problem, and it should have been made clear. I was asked on the app if I needed help.",Critique/Other
2575,"as someone that went from ~4k followers to what i've got now, i've seen a huge difference.",Critique/Other
39507,"Schools can't force kids not to vape in school, not to do drugs in school, not to bully other students, can't convince them to do their schoolwork, wear school appropriate clothing...I work at a high school. Discipline is a constant, ongoing process.",Critique/Other
10538,Miley Cyrus shocks fans with date-rape 'joke' during gig at GAY in London,Critique/Other
35311,Disney High School Bullies be like :,Critique/Other
...,...,...
47556,great :D! which one is ur first language,Critique/Other
13069,Oh definitely. The anime community definitely pick apart female character way more than males ones. Already seen some anime onlies call Echidna a bitch and she hasn’t done anything yet. Not to forget the king candidates when they refused to help Subaru in ...,Critique/Other
40567,"Nigger unchained...“: Epitome of ""nigger"" CheesusSlice: Maurice Clarett dumb as fuck”",Insults
15599,pitch perfect parody. My bad.,Critique/Other
